# Reproduce the formulation and resolution of Model Predictive Control as a QP problem

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from qpsolvers import solve_qp



In [25]:
class MPCParams():
    def __init__(self, T, N, h_CoM, g):
        self.T = T
        self.N = N
        self.h_CoM = h_CoM
        self.g = g
        self.Px = np.array([[1., (i+1)*T, (i+1)**2*T**2/2 - h_CoM/g] for i in range(N)])
        self.Pu = np.zeros((N,N))
        for i in range(N):
            for j in range(i+1):
                self.Pu[i, i-j] = (1 + 3*j + 3*(j**2)) * (T**3/6) - T*h_CoM/g

        self.A = np.array([[1., T, (T**2)/2], [0., 1., T], [0., 0., 1.]]) 
        self.b = np.array([(T**3)/6., T**2/2., T])

        self.e = np.array([1., 0., h_CoM/g])
        
        # initial x
        self.x_dim = 3
        self.x = np.zeros(self.x_dim)

        # output
        self.jerk_dim = N

    def compute_next_x(self):
        return self.A @ self.x + self.jerk[-1] * self.b
    
    def solve(self,
              Zmin: np.array,
              Zmax: np.array):
        
        self.P = np.identity(self.N)
        self.q = np.zeros(self.N)
        self.G = np.vstack([self.Pu, -self.Pu])
        self.h = np.hstack([Zmax - self.Px @ self.x, -Zmin + self.Px @ self.x])
        jerk = solve_qp(self.P, self.q, self.G, self.h, solver='daqp')
        if jerk is None:
            raise ValueError("Not valid solution QP")
        self.jerk = jerk
        self.x = self.compute_next_x()
        self.z = self.e @ self.x

In [26]:
def create_z_limit_values(samples_per_value):
    values_min = [-.175, -.175, .025, -.175, .05, -.15, .075, -.15]
    values_max = [.175, -.025, .175, .025, .2, 0.0, 0.2, 0.2]
    z_min =  np.array([v for v in values_min for _ in range(samples_per_value)])
    z_max =  np.array([v for v in values_max for _ in range(samples_per_value)])
    return z_min, z_max

T = 0.3
jerk = 0.01
g = 9.87
h_CoM = 1
Ni = 15  # Samples per step
total_data = Ni * 8
N = 10

Zmin, Zmax = create_z_limit_values(Ni)
mpc = MPCParams(T, N, h_CoM, g)
x_path = []
z_path = []
last_index = 0
for k in range(total_data//N):
    zmin_N = Zmin[last_index:last_index+N]
    zmax_N = Zmax[last_index:last_index+N]
    mpc.solve(zmin_N, zmax_N)
    x_path.append(mpc.x[0])
    z_path.append(mpc.z)
    last_index = last_index + N


ValueError: Not valid solution QP

In [28]:
plt.plot(Zmin,'-o', label= 'Zmin')
plt.plot(z_path,'-o', label="Z")
plt.plot(x_path,'-o', label="x")
plt.plot(Zmax,'-o', label="Zmax")
plt.legend()
plt.ylim(-1,1)

(-1.0, 1.0)

Error in callback <function _draw_all_if_interactive at 0x0000021D64B98820> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: In FT2Font: Can not load face (invalid stream operation; error code 0x55)

RuntimeError: In FT2Font: Can not load face (invalid stream operation; error code 0x55)

<Figure size 640x480 with 1 Axes>